In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np

from data_loader import DataLoader
from optim import Adam
from rnn import RNN, rnn_training_step, sample, init_params

In [3]:
seq_length = 25
with open('data/input.txt') as fp:
    data_loader = DataLoader(fp.read(), batch_size=seq_length)

In [4]:
hidden_size = 100
num_epochs = 2
sample_every = 10000
sample_size = 200
print_every = 10000

rnn = RNN()
params = init_params(data_loader.vocab_size, hidden_size=hidden_size)
optimizer = Adam(params)
it = 0
for epoch in range(num_epochs):
    hidden_state = np.zeros((1, hidden_size))
    for x, y in data_loader:
        if it % sample_every == 0:
            one_hot = sample(rnn, hidden_state, x[0], params, sample_size)
            generated_text = data_loader.decode(one_hot)
            print()
            print(generated_text)
            print()
        loss, hidden_state, dparams = rnn_training_step(rnn, hidden_state, x, y, params)
        if it % print_every == 0:
            print('iteration: {}, loss: {}'.format(it, loss))
        optimizer.step(dparams)
        it += 1


nnden;:ibt 3n:o:'vj?tb-lgp
sai&3 $nd-ivn-:ztm.x.;3ih zcd&ahb h
$:dle:.fd, yl..fe'ri.or,l.ubypqw&y?q:ps.b!g!!li$,q3nrpxs'ce,ko-.:?l'eqoplvcvcs;'vm,mvcbjj&zac;uhzqxd:sqzaf!k&cfju,
nq,p3lpr$$.dlk.gr!-hnd

iteration: 0, loss: 3.6634675465625888

 of whem aint fok of haw,
and stil?
:
bece for make fin is you thels nob, with to netelind 't in my liste.

satitess'd.

by ard noml,
by makinof whe pikent int hather then andinnd gishisethatcand your

iteration: 10000, loss: 2.16412378537847

?

jurece will:
butthot streich it hat with
stermed, there of sa cauls:
wold atap cow a tecplioft'me she same'st whastewile iner mysenpoust sinere.

durso!

rupfow!--ofe.

rumes!

nomeas sattn my is.


iteration: 20000, loss: 1.7250549474268468

s:
and will to truck bit with that intast forchmand-sight oon,
you hend canderve, that that' home i isien, heme? whing his measingn whis is his kintse with thou notwacay youn:
near'd than it,
'tes:
th

iteration: 30000, loss: 1.9739785540711368

s him sea! these offi